# Text-to-SQL 프롬프트 템플릿 생성

이 노트북은 자연어 질문을 SQL 쿼리로 변환하기 위한 프롬프트 템플릿을 생성하고 관리하는 전용 모듈입니다.

## 주요 기능
- 데이터베이스 스키마를 프롬프트용 컨텍스트로 변환
- Few-shot learning을 위한 예시 포함
- SQL 쿼리 추출 및 검증 함수
- 다양한 질문 유형에 대한 템플릿 제공

## VS Code에서 Databricks 사용하기

VS Code에서도 Databricks Workspace처럼 작업할 수 있습니다! 

### 🚀 VS Code Databricks Extension 설정

1. **확장 설치**: VS Code에서 "Databricks" 확장 설치
2. **클러스터 연결**: 직접 Databricks 클러스터에 연결 가능
3. **노트북 실행**: 이 노트북을 그대로 Databricks 클러스터에서 실행
4. **실시간 개발**: 로컬 파일을 Databricks 환경에서 바로 테스트

### 🔧 설정 방법
```bash
# VS Code에서 Databricks 확장 설치 후
# 1. Ctrl+Shift+P → "Databricks: Configure Workspace"
# 2. Databricks 워크스페이스 URL 입력
# 3. 개인 액세스 토큰 입력
# 4. 클러스터 선택 및 연결
```

### ✨ 장점
- **로컬 개발**: 로컬에서 코드 작성 후 Databricks에서 실행
- **버전 관리**: Git과 완벽 통합
- **IntelliSense**: 코드 자동완성 및 디버깅
- **동기화**: 로컬 파일과 Databricks 워크스페이스 동기화

## 1. 필요한 라이브러리 임포트

In [ ]:
from langchain.prompts import PromptTemplate
import re
import os

## 2. 환경 설정 및 확인

In [ ]:
import os

# 현재 환경 확인 - VS Code Databricks Extension 지원
is_databricks = "DATABRICKS_RUNTIME_VERSION" in os.environ
is_vscode_databricks = False

# VS Code Databricks Extension 확인
try:
    import pyspark
    from pyspark.sql import SparkSession
    
    # Spark 세션 생성 시도
    spark = SparkSession.builder.appName("VSCode-Databricks").getOrCreate()
    
    # Databricks 특정 기능 확인
    if hasattr(spark, 'sql') and not is_databricks:
        is_vscode_databricks = True
        print("🔥 VS Code Databricks Extension 환경 감지됨!")
        print("   - 로컬 VS Code에서 Databricks 클러스터에 연결됨")
        
except Exception as e:
    spark = None
    print("⚠️ Spark 연결 실패:", str(e)[:50])

# 환경 상태 출력
if is_databricks:
    print("✅ Native Databricks 환경")
    environment = "databricks"
elif is_vscode_databricks:
    print("✅ VS Code + Databricks Extension 환경")
    environment = "vscode_databricks"
else:
    print("✅ 로컬 개발 환경")
    environment = "local"

print(f"🔧 환경: {environment}")
print(f"🔗 Spark 세션: {'활성화' if spark else '비활성화'}")

# 환경별 기능 가용성 체크
features = {
    "vector_search": is_databricks or is_vscode_databricks,
    "spark_sql": bool(spark),
    "databricks_llm": is_databricks or is_vscode_databricks,
    "local_embedding": not (is_databricks or is_vscode_databricks)
}

print("\n📋 사용 가능한 기능:")
for feature, available in features.items():
    status = "✅" if available else "❌"
    print(f"   {status} {feature}")

## 3. 스키마 컨텍스트 생성 함수

In [ ]:
def create_schema_context(db_schema):
    """데이터베이스 스키마를 프롬프트용 텍스트로 변환
    
    Args:
        db_schema (list): 데이터베이스 스키마 정보 리스트
    
    Returns:
        str: 프롬프트에 사용할 스키마 컨텍스트 텍스트
    """
    schema_text = "데이터베이스 스키마 정보:\n\n"
    
    for table_info in db_schema:
        schema_text += f"테이블: {table_info['table_name']}\n"
        schema_text += "컬럼:\n"
        
        for col in table_info['columns']:
            comment = f" -- {col['comment']}" if col.get('comment') else ""
            schema_text += f"  - {col['name']} ({col['type']}){comment}\n"
        
        # 샘플 데이터 추가
        if table_info['sample_data']:
            schema_text += "샘플 데이터:\n"
            for i, sample in enumerate(table_info['sample_data'][:2]):
                schema_text += f"  {sample}\n"
        
        schema_text += "\n"
    
    return schema_text

print("✅ 스키마 컨텍스트 생성 함수 정의 완료")

## 4. SQL 추출 함수

In [ ]:
def extract_sql_from_response(response_text):
    """LLM 응답에서 SQL 쿼리만 추출
    
    Args:
        response_text (str): LLM의 전체 응답 텍스트
    
    Returns:
        str: 추출된 SQL 쿼리
    """
    # ```sql과 ``` 사이의 내용 추출
    sql_pattern = r'```sql\s*(.*?)\s*```'
    matches = re.findall(sql_pattern, response_text, re.DOTALL | re.IGNORECASE)
    
    if matches:
        return matches[0].strip()
    
    # 백틱이 없는 경우 전체 응답을 SQL로 간주 (간단한 경우)
    return response_text.strip()

def validate_sql_query(sql_query):
    """기본적인 SQL 쿼리 검증
    
    Args:
        sql_query (str): 검증할 SQL 쿼리
    
    Returns:
        tuple: (is_valid, error_message)
    """
    if not sql_query.strip():
        return False, "빈 쿼리입니다."
    
    # 기본적인 SQL 키워드 확인
    sql_keywords = ['SELECT', 'INSERT', 'UPDATE', 'DELETE', 'CREATE', 'DROP', 'ALTER']
    upper_query = sql_query.upper()
    
    if not any(keyword in upper_query for keyword in sql_keywords):
        return False, "유효한 SQL 키워드가 없습니다."
    
    # 기본적인 구문 검증 (괄호 매칭)
    open_parens = sql_query.count('(')
    close_parens = sql_query.count(')')
    
    if open_parens != close_parens:
        return False, "괄호가 일치하지 않습니다."
    
    return True, "유효한 SQL 쿼리입니다."

print("✅ SQL 추출 및 검증 함수 정의 완료")

## 5. Text-to-SQL 프롬프트 템플릿

In [ ]:
# 기본 Text-to-SQL 프롬프트 템플릿
basic_sql_prompt_template = """
당신은 SQL 쿼리 생성 전문가입니다. 주어진 데이터베이스 스키마를 바탕으로 자연어 질문을 정확한 SQL 쿼리로 변환하세요.

{schema_context}

규칙:
1. 정확한 테이블명과 컬럼명을 사용하세요
2. 적절한 JOIN을 사용하세요
3. WHERE 절을 적절히 활용하세요
4. 결과는 SQL 쿼리만 반환하세요 (설명 없이)
5. 쿼리는 ```sql과 ``` 사이에 작성하세요

예시:
질문: "모든 고객의 이름과 이메일을 보여주세요"
답변:
```sql
SELECT name, email FROM customers;
```

질문: "2024년 1월에 주문한 고객들의 총 주문 금액을 계산해주세요"
답변:
```sql
SELECT c.name, SUM(o.amount) as total_amount
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date >= '2024-01-01' AND o.order_date < '2024-02-01'
GROUP BY c.customer_id, c.name;
```

질문: {question}
답변:
"""

print("✅ 기본 Text-to-SQL 프롬프트 템플릿 정의 완료")

## 6. 고급 프롬프트 템플릿 (한국어 특화)

In [ ]:
# 한국어 특화 고급 프롬프트 템플릿
advanced_korean_sql_prompt_template = """
당신은 한국어를 이해하는 SQL 전문가입니다. 주어진 데이터베이스 스키마를 바탕으로 한국어 질문을 정확한 SQL 쿼리로 변환하세요.

{schema_context}

변환 규칙:
1. 테이블명과 컬럼명을 정확히 사용하세요
2. 한국어 숫자 표현을 숫자로 변환하세요 (예: "열 개" → 10)
3. 날짜 표현을 SQL 형식으로 변환하세요 (예: "작년" → 해당 연도)
4. 집계 함수를 적절히 사용하세요
5. 결과는 SQL 쿼리만 반환하고 ```sql 블록으로 감싸세요

한국어 질문 예시:

질문: "고객 수를 세어주세요"
```sql
SELECT COUNT(*) as customer_count FROM customers;
```

질문: "가장 많이 주문한 고객 다섯 명을 찾아주세요"
```sql
SELECT c.name, COUNT(o.order_id) as order_count
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name
ORDER BY order_count DESC
LIMIT 5;
```

질문: "월별 매출을 보여주세요"
```sql
SELECT 
    DATE_FORMAT(order_date, '%Y-%m') as month,
    SUM(amount) as monthly_revenue
FROM orders
GROUP BY DATE_FORMAT(order_date, '%Y-%m')
ORDER BY month;
```

질문: "{question}"
답변:
"""

print("✅ 한국어 특화 고급 프롬프트 템플릿 정의 완료")

## 7. 프롬프트 템플릿 생성 함수

In [ ]:
def create_sql_prompt_template(db_schema, template_type="basic"):
    """SQL 생성을 위한 프롬프트 템플릿을 생성
    
    Args:
        db_schema (list): 데이터베이스 스키마 정보
        template_type (str): 템플릿 유형 ('basic' 또는 'advanced')
    
    Returns:
        PromptTemplate: LangChain 프롬프트 템플릿
    """
    # 스키마 컨텍스트 생성
    schema_context = create_schema_context(db_schema)
    
    # 템플릿 선택
    if template_type == "advanced":
        template = advanced_korean_sql_prompt_template
    else:
        template = basic_sql_prompt_template
    
    # 프롬프트 템플릿 생성
    sql_prompt = PromptTemplate(
        template=template,
        input_variables=["question"],
        partial_variables={"schema_context": schema_context}
    )
    
    return sql_prompt

def create_schema_summary(db_schema):
    """데이터베이스 스키마 요약 정보 생성
    
    Args:
        db_schema (list): 데이터베이스 스키마 정보
    
    Returns:
        dict: 스키마 요약 정보
    """
    summary = {
        "total_tables": len(db_schema),
        "tables": [],
        "total_columns": 0
    }
    
    for table_info in db_schema:
        table_summary = {
            "name": table_info['table_name'],
            "column_count": len(table_info['columns']),
            "columns": [col['name'] for col in table_info['columns']],
            "has_sample_data": bool(table_info.get('sample_data'))
        }
        summary["tables"].append(table_summary)
        summary["total_columns"] += table_summary["column_count"]
    
    return summary

print("✅ 프롬프트 템플릿 생성 함수 정의 완료")

## 8. 테스트용 샘플 스키마

In [ ]:
# 테스트용 샘플 스키마 정의
sample_db_schema = [
    {
        'table_name': 'customers',
        'columns': [
            {'name': 'customer_id', 'type': 'int', 'comment': '고객 고유 식별자'},
            {'name': 'name', 'type': 'string', 'comment': '고객명'},
            {'name': 'email', 'type': 'string', 'comment': '이메일 주소'},
            {'name': 'phone', 'type': 'string', 'comment': '전화번호'},
            {'name': 'address', 'type': 'string', 'comment': '주소'},
            {'name': 'created_at', 'type': 'timestamp', 'comment': '계정 생성일시'},
            {'name': 'status', 'type': 'string', 'comment': '고객 상태 (active, inactive)'}
        ],
        'sample_data': [
            {'customer_id': 1, 'name': '김철수', 'email': 'kim@example.com', 'status': 'active'},
            {'customer_id': 2, 'name': '이영희', 'email': 'lee@example.com', 'status': 'active'}
        ]
    },
    {
        'table_name': 'orders',
        'columns': [
            {'name': 'order_id', 'type': 'int', 'comment': '주문 고유 식별자'},
            {'name': 'customer_id', 'type': 'int', 'comment': '고객 ID (customers 테이블 참조)'},
            {'name': 'product_name', 'type': 'string', 'comment': '상품명'},
            {'name': 'quantity', 'type': 'int', 'comment': '주문 수량'},
            {'name': 'unit_price', 'type': 'decimal', 'comment': '단가'},
            {'name': 'total_amount', 'type': 'decimal', 'comment': '총 주문 금액'},
            {'name': 'order_date', 'type': 'date', 'comment': '주문 일자'},
            {'name': 'status', 'type': 'string', 'comment': '주문 상태 (pending, completed, cancelled)'}
        ],
        'sample_data': [
            {'order_id': 1, 'customer_id': 1, 'product_name': '노트북', 'total_amount': 1200000, 'order_date': '2024-01-15'},
            {'order_id': 2, 'customer_id': 2, 'product_name': '마우스', 'total_amount': 50000, 'order_date': '2024-01-16'}
        ]
    },
    {
        'table_name': 'products',
        'columns': [
            {'name': 'product_id', 'type': 'int', 'comment': '상품 고유 식별자'},
            {'name': 'product_name', 'type': 'string', 'comment': '상품명'},
            {'name': 'category', 'type': 'string', 'comment': '상품 카테고리'},
            {'name': 'price', 'type': 'decimal', 'comment': '상품 가격'},
            {'name': 'stock_quantity', 'type': 'int', 'comment': '재고 수량'},
            {'name': 'created_at', 'type': 'timestamp', 'comment': '상품 등록일시'}
        ],
        'sample_data': [
            {'product_id': 1, 'product_name': '노트북', 'category': '전자제품', 'price': 1200000},
            {'product_id': 2, 'product_name': '마우스', 'category': '전자제품', 'price': 50000}
        ]
    }
]

print("✅ 테스트용 샘플 스키마 정의 완료")
print(f"테이블 수: {len(sample_db_schema)}")
for table in sample_db_schema:
    print(f"  - {table['table_name']}: {len(table['columns'])}개 컬럼")

## 9. 프롬프트 템플릿 테스트

In [ ]:
# 기본 프롬프트 템플릿 테스트
print("=== 기본 프롬프트 템플릿 테스트 ===")
basic_prompt = create_sql_prompt_template(sample_db_schema, "basic")

test_question = "모든 고객의 이름과 이메일을 보여주세요"
formatted_prompt = basic_prompt.format(question=test_question)

print("생성된 프롬프트 (처음 500자):")
print(formatted_prompt[:500] + "...")
print(f"\n전체 길이: {len(formatted_prompt)} 문자")

In [ ]:
# 고급 프롬프트 템플릿 테스트
print("=== 고급 프롬프트 템플릿 테스트 ===")
advanced_prompt = create_sql_prompt_template(sample_db_schema, "advanced")

test_question = "가장 많이 주문한 고객 5명을 찾아주세요"
formatted_prompt = advanced_prompt.format(question=test_question)

print("생성된 프롬프트 (처음 500자):")
print(formatted_prompt[:500] + "...")
print(f"\n전체 길이: {len(formatted_prompt)} 문자")

## 10. 스키마 정보 요약

In [ ]:
# 스키마 요약 정보 출력
schema_summary = create_schema_summary(sample_db_schema)

print("=== 데이터베이스 스키마 요약 ===")
print(f"총 테이블 수: {schema_summary['total_tables']}")
print(f"총 컬럼 수: {schema_summary['total_columns']}")

print("\n테이블별 상세 정보:")
for table in schema_summary['tables']:
    print(f"\n📊 {table['name']}")
    print(f"   컬럼 수: {table['column_count']}")
    print(f"   컬럼 목록: {', '.join(table['columns'])}")
    print(f"   샘플 데이터: {'있음' if table['has_sample_data'] else '없음'}")

## 11. 유틸리티 함수들

In [ ]:
def preview_schema_context(db_schema, max_length=1000):
    """스키마 컨텍스트 미리보기
    
    Args:
        db_schema (list): 데이터베이스 스키마 정보
        max_length (int): 최대 출력 길이
    """
    context = create_schema_context(db_schema)
    
    if len(context) > max_length:
        preview = context[:max_length] + "\n... (내용이 잘렸습니다)"
    else:
        preview = context
    
    print("=== 스키마 컨텍스트 미리보기 ===")
    print(preview)
    print(f"\n전체 길이: {len(context)} 문자")

def test_sql_extraction():
    """SQL 추출 함수 테스트"""
    test_responses = [
        "```sql\nSELECT * FROM customers;\n```",
        "여기 SQL 쿼리입니다:\n```sql\nSELECT name, email FROM customers WHERE status = 'active';\n```\n이 쿼리는 활성 고객을 조회합니다.",
        "SELECT COUNT(*) FROM orders;"
    ]
    
    print("=== SQL 추출 함수 테스트 ===")
    for i, response in enumerate(test_responses, 1):
        extracted = extract_sql_from_response(response)
        is_valid, message = validate_sql_query(extracted)
        
        print(f"\n테스트 {i}:")
        print(f"원본: {response[:50]}...")
        print(f"추출된 SQL: {extracted}")
        print(f"검증 결과: {'✅' if is_valid else '❌'} {message}")

# 함수 테스트 실행
preview_schema_context(sample_db_schema)
test_sql_extraction()

print("\n✅ 모든 유틸리티 함수 테스트 완료!")

## 12. 모듈 사용 예시

In [ ]:
# 이 모듈을 다른 노트북에서 사용하는 방법
usage_example = """
# 다른 노트북에서 이 모듈 사용하기

# 1. 필요한 함수들 임포트 (또는 전체 노트북 실행)
# %run text_to_sql_prompt_template.ipynb

# 2. 데이터베이스 스키마 준비 (실제 스키마 또는 샘플 사용)
my_schema = sample_db_schema  # 또는 실제 스키마

# 3. 프롬프트 템플릿 생성
sql_prompt = create_sql_prompt_template(my_schema, "advanced")

# 4. 질문으로 프롬프트 포맷팅
question = "월별 매출을 보여주세요"
formatted_prompt = sql_prompt.format(question=question)

# 5. LLM에 전달하여 SQL 생성
# response = llm.predict(formatted_prompt)
# sql_query = extract_sql_from_response(response)

# 6. SQL 검증
# is_valid, message = validate_sql_query(sql_query)
"""

print("=== 모듈 사용 예시 ===")
print(usage_example)

print("\n🎉 Text-to-SQL 프롬프트 템플릿 모듈 준비 완료!")
print("이 노트북은 다른 RAG 시스템에서 재사용할 수 있습니다.")

# 🚀 VS Code Databricks Extension에서 이 모듈 사용하기

print("=== VS Code + Databricks Extension 환경에서 사용법 ===\n")

# 1. VS Code Databricks Extension 설정 가이드
vscode_setup_guide = """
🔧 VS Code Databricks Extension 설정:

1. 확장 설치:
   - VS Code에서 'Databricks' 검색하여 설치
   - Microsoft의 공식 Databricks 확장 사용

2. 워크스페이스 연결:
   - Ctrl+Shift+P → "Databricks: Configure Workspace"
   - Databricks URL 입력: https://your-workspace.cloud.databricks.com
   - 개인 액세스 토큰(PAT) 입력

3. 클러스터 연결:
   - 좌측 Databricks 패널에서 클러스터 선택
   - "Connect" 클릭하여 활성화

4. 노트북 실행:
   - 이 .ipynb 파일을 바로 Databricks 클러스터에서 실행
   - 셀별로 실행하면 Databricks 환경에서 처리됨
"""

# 2. 실제 사용 예시 코드
usage_code = '''
# VS Code에서 Databricks 클러스터 연결 후 실행 가능한 코드:

# 환경 확인
if is_vscode_databricks:
    print("VS Code + Databricks Extension 환경에서 실행 중!")
    
    # 실제 데이터베이스 스키마 가져오기
    tables = spark.sql("SHOW TABLES").collect()
    print(f"사용 가능한 테이블: {len(tables)}개")
    
    # 실제 스키마로 프롬프트 생성
    if tables:
        # 실제 테이블 정보를 사용한 스키마 구성
        real_schema = []
        for table in tables[:3]:  # 처음 3개 테이블만
            table_name = table.tableName
            columns_df = spark.sql(f"DESCRIBE TABLE {table_name}")
            columns = [{"name": row.col_name, "type": row.data_type} 
                      for row in columns_df.collect()]
            real_schema.append({
                "table_name": table_name,
                "columns": columns,
                "sample_data": []
            })
        
        # 실제 스키마로 프롬프트 생성
        real_prompt = create_sql_prompt_template(real_schema, "advanced")
        print("✅ 실제 데이터베이스 스키마 기반 프롬프트 생성 완료!")
'''

# 3. 장점 및 특징
advantages = """
🌟 VS Code + Databricks Extension의 장점:

✅ 로컬 개발 경험:
   - 익숙한 VS Code 인터페이스
   - Git 버전 관리 완벽 지원
   - 로컬 파일 편집 후 원격 실행

✅ 실시간 협업:
   - 팀원과 코드 공유 용이
   - 로컬에서 개발 → Databricks에서 테스트

✅ 하이브리드 개발:
   - 로컬 테스트용 작은 데이터셋
   - 실제 운영은 Databricks 클러스터

✅ 디버깅 및 개발 도구:
   - VS Code의 강력한 디버깅 기능
   - IntelliSense 자동완성
   - 확장된 터미널 기능
"""

print(vscode_setup_guide)
print("\n" + "="*60)
print(usage_code)
print("\n" + "="*60)
print(advantages)

print("\n🎯 현재 환경에서 추천 워크플로우:")
if is_vscode_databricks:
    print("✨ VS Code Extension 환경 - 모든 기능 사용 가능!")
    print("   1. 이 노트북을 그대로 실행")
    print("   2. 실제 Databricks 테이블 스키마 사용")
    print("   3. Databricks LLM 모델 직접 호출")
elif is_databricks:
    print("✨ Native Databricks 환경 - 최적화된 성능!")
    print("   1. 모든 Databricks 기능 활용")
    print("   2. Vector Search 및 ML 파이프라인")
elif environment == "local":
    print("💡 로컬 환경 - 개발 및 테스트용:")
    print("   1. 샘플 스키마로 프롬프트 테스트")
    print("   2. OpenAI API 사용한 SQL 생성")
    print("   3. VS Code Databricks Extension 설치 고려")

print(f"\n🔧 환경 설정 완료! ({environment})")
print("🚀 Text-to-SQL 시스템 준비 완료!")